In [43]:
import pymysql
import pandas as df
from pandas import *
import json
import re
import jieba
from numpy import *
from numpy import linalg as la 

In [114]:
#连接数据库
conn = pymysql.connect('localhost','root','9432','companyb',charset='utf8')
cursor=conn.cursor()

In [115]:
cursor.execute("select * from (select investor,GROUP_CONCAT(industry) as allind from com_inv_rel as a GROUP BY investor) as t where allind is not null order by investor")
a=cursor.fetchall()

In [12]:
indlist=[]
for i in range(len(a)):
    indlist+=a[i][1].split(',')
print(len(indlist))
indlist=sorted(list(set(indlist)))
print(len(indlist))

304382
93


In [116]:
invlist=[a[i][0] for i in range(len(a))]

In [28]:
df=DataFrame(np.zeros((len(a),len(indlist))),columns=indlist)

In [29]:
for i in range(len(a)):
    ilist=a[i][1].split(',')
    for ic in ilist:
        df.loc[i,ic]+=1

In [32]:
df.to_csv('InvestIndustry.csv',index=False)

In [109]:
df=read_csv('InvestIndustry.csv',encoding='gbk')

In [21]:
#求余弦
def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

#先求整体相似矩阵
def similarMat(dataMat,simMeas=cosSim):
    U,Sigma,VT = la.svd(dataMat,False)
    Sig2=Sigma**2
    for i in range(Sig2.shape[0]): #找到达到90能量的奇异值数目
        if sum(Sig2[:i]) > sum(Sig2)*0.9:
            s=i
            break
    n = shape(dataMat)[1]
    m = shape(dataMat)[0]
    M=np.zeros((n,n))
    Sig = mat(eye(s)*Sigma[:s]) #arrange Sig into a diagonal matrix  
    xformedItems = dataMat.T * U[:,:s] * Sig.I  #create transformed items  
    for i in range(n):
        for j in range(i+1,n):  
            similarity = simMeas(xformedItems[i,:].T,xformedItems[j,:].T)
            M[i,j]=similarity
            M[j,i]=similarity
            print('the %d and %d similarity is: %f'%(i, j, similarity))
    return M

def svdEst(dataMat, user, simMat, item):
    simTotal = 0.0; ratSimTotal = 0.0;rateTotal=0.0
    n = shape(dataMat)[1]
    for j in range(n): 
        userRating = dataMat[user,j]  
        if userRating == 0 or j==item: continue  
        similarity = simMat[item,j]
        #simTotal += similarity  
        rateTotal+=userRating
        ratSimTotal += similarity * userRating  
    if rateTotal == 0: return 0  
    else: return ratSimTotal/rateTotal
#simMat=similarMat(mat(A)) #求出相似矩阵
#按照用户推荐
def recommend(dataMat,user,simMat,N=3,simMeas=cosSim,estMethod=svdEst):
    #寻找未评级的物品  
    unratedItems=nonzero(dataMat[user,:].A==0)[1]  
    if len(unratedItems)==0: return 'you rated everything'  
    itemScores=[]  
    for item in unratedItems: 
        estimatedScore=estMethod(dataMat,user,simMat,item) #对每一个未评分物品，调用standEst()来产生该物品的预测得分  
        itemScores.append((item,estimatedScore)) #该物品的编号和估计得分值放入一个元素列表itemScores中  
    #对itemScores进行从大到小排序，返回前N个未评分物品  
    return sorted(itemScores,key=lambda jj:jj[1],reverse=True)[:N]  

In [9]:
A=df.as_matrix()
simMat=similarMat(mat(A))

the 0 and 1 similarity is: 0.499952
the 0 and 2 similarity is: 0.506003
the 0 and 3 similarity is: 0.529409
the 0 and 4 similarity is: 0.515426
the 0 and 5 similarity is: 0.571826
the 0 and 6 similarity is: 0.528516
the 0 and 7 similarity is: 0.526340
the 0 and 8 similarity is: 0.543688
the 0 and 9 similarity is: 0.499920
the 0 and 10 similarity is: 0.523369
the 0 and 11 similarity is: 0.499914
the 0 and 12 similarity is: 0.559138
the 0 and 13 similarity is: 0.499971
the 0 and 14 similarity is: 0.498974
the 0 and 15 similarity is: 0.499965
the 0 and 16 similarity is: 0.499955
the 0 and 17 similarity is: 0.499932
the 0 and 18 similarity is: 0.510077
the 0 and 19 similarity is: 0.499929
the 0 and 20 similarity is: 0.499982
the 0 and 21 similarity is: 0.522322
the 0 and 22 similarity is: 0.499975
the 0 and 23 similarity is: 0.468056
the 0 and 24 similarity is: 0.499949
the 0 and 25 similarity is: 0.499608
the 0 and 26 similarity is: 0.532018
the 0 and 27 similarity is: 0.514883
the 0 and 

the 12 and 71 similarity is: 0.659279
the 12 and 72 similarity is: 0.618039
the 12 and 73 similarity is: 0.477335
the 12 and 74 similarity is: 0.520894
the 12 and 75 similarity is: 0.535676
the 12 and 76 similarity is: 0.498316
the 12 and 77 similarity is: 0.476384
the 12 and 78 similarity is: 0.708372
the 12 and 79 similarity is: 0.658184
the 12 and 80 similarity is: 0.652085
the 12 and 81 similarity is: 0.630494
the 12 and 82 similarity is: 0.523419
the 12 and 83 similarity is: 0.641089
the 12 and 84 similarity is: 0.572430
the 12 and 85 similarity is: 0.763238
the 12 and 86 similarity is: 0.552231
the 12 and 87 similarity is: 0.674758
the 12 and 88 similarity is: 0.867593
the 12 and 89 similarity is: 0.615345
the 12 and 90 similarity is: 0.537040
the 12 and 91 similarity is: 0.748528
the 12 and 92 similarity is: 0.796674
the 13 and 14 similarity is: 0.520208
the 13 and 15 similarity is: 0.499915
the 13 and 16 similarity is: 0.499760
the 13 and 17 similarity is: 0.499931
the 13 and 1

the 45 and 89 similarity is: 0.645869
the 45 and 90 similarity is: 0.499781
the 45 and 91 similarity is: 0.694157
the 45 and 92 similarity is: 0.558649
the 46 and 47 similarity is: 0.523211
the 46 and 48 similarity is: 0.513608
the 46 and 49 similarity is: 0.501488
the 46 and 50 similarity is: 0.641613
the 46 and 51 similarity is: 0.578303
the 46 and 52 similarity is: 0.528410
the 46 and 53 similarity is: 0.535251
the 46 and 54 similarity is: 0.528590
the 46 and 55 similarity is: 0.535354
the 46 and 56 similarity is: 0.502747
the 46 and 57 similarity is: 0.499698
the 46 and 58 similarity is: 0.481405
the 46 and 59 similarity is: 0.556759
the 46 and 60 similarity is: 0.654453
the 46 and 61 similarity is: 0.536280
the 46 and 62 similarity is: 0.500342
the 46 and 63 similarity is: 0.488923
the 46 and 64 similarity is: 0.505535
the 46 and 65 similarity is: 0.498943
the 46 and 66 similarity is: 0.510777
the 46 and 67 similarity is: 0.534847
the 46 and 68 similarity is: 0.493747
the 46 and 6

In [38]:
indlist=df.columns.tolist()

In [139]:
com=138473
print(invlist[com])
print("已投行业：")
comind=df.loc[com,:].tolist()
for i in range(len(comind)):
    if comind[i]!=0:
        print(indlist[i],comind[i])
res=recommend(mat(A),com,simMat,N=5)
result=[]
print('推荐：')
for r in res:
    print(indlist[r[0]],r[1])

江苏福瑞至环保新材料有限责任公司
已投行业：
科技推广和应用服务业 1.0
推荐：
互联网和相关服务 0.63541552713
电信、广播电视和卫星传输服务 0.626650505298
新闻和出版业 0.567697167544
教育 0.542460148407
体育 0.53870354963


In [41]:
np.save('simMat',simMat)

In [44]:
simMat.shape

(93, 93)

In [144]:
from pyecharts import Graph
nodes=[{"name":indlist[i],"symbolsize":10} for i in range(len(indlist))]
vlist=[]
links=[]
for i in range(simMat.shape[0]):
    for j in range(i+1,simMat.shape[0]):
        if simMat[i][j]>0.7:
            links.append({"source":indlist[i],"target":indlist[j],"value":simMat[i][j]})
            vlist.append(simMat[i][j])

In [145]:
len(vlist)

302

In [146]:
graph = Graph("行业相关性")
graph.add("", nodes, links[:],graph_edge_length=vlist[:],is_label_show=True)
graph

In [76]:
#neo4j
from py2neo import *
neograph = Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="sao6039506"
)

In [105]:
neograph.delete_all()

In [106]:
for i in indlist:
    node=Node("industry",name=i)
    neograph.create(node)

In [ ]:
for i in range(simMat.shape[0]):
    for j in range(i+1,simMat.shape[0]):
        print(i,j)
        n1=neograph.find_one("industry",property_key="name",property_value=indlist[i])
        n2=neograph.find_one("industry",property_key="name",property_value=indlist[j])
        rel1 = Relationship(n1,'rel',n2)
        rel1['value']=simMat[i][j]
        rel2 = Relationship(n2,'rel',n1)
        rel2['value']=simMat[i][j]
        neograph.create(rel1)
        neograph.create(rel2)

In [90]:
neograph.create(node)

In [110]:
df

,专业技术服务业,专用设备制造业,中国共产党机关,互联网和相关服务,仓储业,仪器仪表制造业,住宿业,体育,保险业,公共设施管理业,...,金属制品业,铁路、船舶、航空航天和其他运输设备制造业,铁路运输业,零售业,非金属矿物制品业,非金属矿采选业,食品制造业,餐饮业,黑色金属冶炼和压延加工业,黑色金属矿采选业
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
